In [1]:
#Import the required libraries
!pip install keras-tuner
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import pathlib, os, random
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow as tf
from PIL import Image
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import time
import keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import Sequential
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler


import cv2
import matplotlib.pyplot as plt 
import seaborn as sns
import os
from PIL import Image
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import load_img
from keras.utils import np_utils

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 6.9 MB/s 
     |████████████████████████████████| 1.6 MB 40.0 MB/s 


In [2]:
#Understanding the files in different data directories
bad_data = os.listdir('/content/drive/MyDrive/Ethiopia_2/Bad')
print("Bad coffee beans: ",bad_data[:10])

good_data = os.listdir('/content/drive/MyDrive/Ethiopia_2/Good')
print("Good coffee beans: ",good_data[:10])

Bad coffee beans:  ['IMG_5979_9.jpg', 'IMG_5980_2.jpg', 'IMG_5980_6.jpg', 'IMG_5980_7.jpg', 'IMG_5979_4.jpg', 'IMG_5979_1.jpg', 'IMG_5979_5.jpg', 'IMG_5979_7.jpg', 'IMG_5979_3.jpg', 'IMG_5980_1.jpg']
Good coffee beans:  ['IMG_5917_8.jpg', 'IMG_5917_3.jpg', 'IMG_5917_7.jpg', 'IMG_5917_9.jpg', 'IMG_5917_5.jpg', 'IMG_5917_2.jpg', 'IMG_5918_1.jpg', 'IMG_5917_1.jpg', 'IMG_5918_5.jpg', 'IMG_5917_6.jpg']


In [3]:
data = []
labels = []
for img in bad_data:
    try:
        img_read = plt.imread('/content/drive/MyDrive/Ethiopia_2/Bad' + "/" + img)
        img_resize = cv2.resize(img_read, (256,256))
        img_array = img_to_array(img_resize)
        data.append(img_array)
        labels.append(1)
    except:
        None
        
for img in good_data:
    try:
        img_read = plt.imread('/content/drive/MyDrive/Ethiopia_2/Good' + "/" + img)
        img_resize = cv2.resize(img_read, (256,256))
        img_array = img_to_array(img_resize)
        data.append(img_array)
        labels.append(0)
    except:
        None

In [4]:
image_data = np.array(data)
labels = np.array(labels)

idx = np.arange(image_data.shape[0])
np.random.shuffle(idx)
image_data = image_data[idx]
labels = labels[idx]

In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(image_data, labels, test_size = 0.2, random_state = 101)

y_train = np_utils.to_categorical(y_train, num_classes = 2)
y_test = np_utils.to_categorical(y_test, num_classes = 2)

print(f'SHAPE OF TRAINING IMAGE DATA : {x_train.shape}')
print(f'SHAPE OF TESTING IMAGE DATA : {x_test.shape}')
print(f'SHAPE OF TRAINING LABELS : {y_train.shape}')
print(f'SHAPE OF TESTING LABELS : {y_test.shape}')

SHAPE OF TRAINING IMAGE DATA : (22, 256, 256, 3)
SHAPE OF TESTING IMAGE DATA : (6, 256, 256, 3)
SHAPE OF TRAINING LABELS : (22, 2)
SHAPE OF TESTING LABELS : (6, 2)


In [6]:
!pip install hyperactive==0.3.1
import hyperactive as hyp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
search_config = {
    "keras.compile.0": {"loss": ["binary_crossentropy"], "optimizer": ["adam"]},
    "keras.fit.0": {"epochs": [5], "batch_size": [32], "verbose": [2]},
    "keras.layers.Conv2D.1": {
        "filters": [32, 64, 128],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
        "input_shape": [(256,256,3)],
    },
    "keras.layers.MaxPooling2D.2": {"pool_size": [(2, 2)]},
    "keras.layers.Conv2D.3": {
        "filters": [16, 32, 64, 128],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
    },
    "keras.layers.MaxPooling2D.4": {"pool_size": [(2, 2)]},
    "keras.layers.Flatten.5": {},
    "keras.layers.Dense.6": {"units": range(30, 100, 10), "activation": ["relu"]},
    "keras.layers.Dropout.7": {"rate": list(np.arange(0.2, 0.8, 0.2))},
    "keras.layers.Dense.8": {"units": [2], "activation": ["softmax"]},
}
start_point = {
    "keras.compile.0": {"loss": ["binary_crossentropy"], "optimizer": ["adam"]},
    "keras.fit.0": {"epochs": [5], "batch_size": [32], "verbose": [2]},
    "keras.layers.Conv2D.1": {
        "filters": [64],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
        "input_shape": [(256,256,3)],
    },
    "keras.layers.MaxPooling2D.2": {"pool_size": [(2, 2)]},
    "keras.layers.Conv2D.3": {
        "filters": [32],
        "kernel_size": [(3, 3)],
        "activation": ["relu"],
    },
    "keras.layers.MaxPooling2D.4": {"pool_size": [(2, 2)]},
    "keras.layers.Flatten.5": {},
    "keras.layers.Dense.6": {"units": [30], "activation": ["relu"]},
    "keras.layers.Dropout.7": {"rate": [0.2]},
    "keras.layers.Dense.8": {"units": [2], "activation": ["softmax"]},
}


#https://pypi.org/project/hyperactive/0.3.1/ 
Optimizer = hyp.ParticleSwarmOptimizer(search_config, n_iter=3,warm_start=start_point, metric="accuracy", n_jobs=1, cv=3, verbosity=1, random_state=42, n_part=3, w=0.85, c_k=1.9, c_s=2.05)

In [8]:
import time
t1 = time.time()
# search best hyperparameter for given data
h = Optimizer.fit(x_train, y_train)

t2 = time.time()
print("time: {}".format(t2-t1))

# predict from test data
Optimizer.predict(x_test)

# calculate accuracy score
score = Optimizer.score(x_test, y_test)

print("test score: {}".format(score))

Epoch 1/5
1/1 - 15s - loss: 20.0147 - accuracy: 0.4545 - 15s/epoch - 15s/step
Epoch 2/5
1/1 - 0s - loss: 1387.6483 - accuracy: 0.5000 - 59ms/epoch - 59ms/step
Epoch 3/5
1/1 - 0s - loss: 749.2117 - accuracy: 0.5000 - 50ms/epoch - 50ms/step
Epoch 4/5
1/1 - 0s - loss: 220.9611 - accuracy: 0.3636 - 50ms/epoch - 50ms/step
Epoch 5/5
1/1 - 0s - loss: 10.6339 - accuracy: 0.5000 - 50ms/epoch - 50ms/step
1/1 [==============================] - 0s 161ms/step - loss: 0.6931 - accuracy: 0.5000


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch 1/5
1/1 - 1s - loss: 50.0458 - accuracy: 0.4091 - 903ms/epoch - 903ms/step
Epoch 2/5
1/1 - 0s - loss: 1849.6711 - accuracy: 0.5000 - 35ms/epoch - 35ms/step
Epoch 3/5
1/1 - 0s - loss: 1175.1753 - accuracy: 0.5000 - 33ms/epoch - 33ms/step
Epoch 4/5
1/1 - 0s - loss: 474.3049 - accuracy: 0.4091 - 33ms/epoch - 33ms/step
Epoch 5/5
1/1 - 0s - loss: 443.1378 - accuracy: 0.3182 - 30ms/epoch - 30ms/step
1/1 [==============================] - 0s 130ms/step - loss: 247.9088 - accuracy: 0.5000
Epoch 1/5
1/1 - 1s - loss: 73.9390 - accuracy: 0.4545 - 775ms/epoch - 775ms/step
Epoch 2/5
1/1 - 0s - loss: 1068.8752 - accuracy: 0.5000 - 50ms/epoch - 50ms/step
Epoch 3/5
1/1 - 0s - loss: 400.7804 - accuracy: 0.5000 - 50ms/epoch - 50ms/step
Epoch 4/5
1/1 - 0s - loss: 109.3046 - accuracy: 0.4545 - 46ms/epoch - 46ms/step
Epoch 5/5
1/1 - 0s - loss: 15.7340 - accuracy: 0.6364 - 46ms/epoch - 46ms/step
1/1 [==============================] - 0s 135ms/step - loss: 10.0354 - accuracy: 0.5000
Epoch 1/5
1/1 - 2s 

 33%|███▎      | 1/3 [00:05<00:11,  5.96s/it]

Epoch 1/5
1/1 - 1s - loss: 16.6832 - accuracy: 0.4091 - 537ms/epoch - 537ms/step
Epoch 2/5
1/1 - 0s - loss: 1898.8013 - accuracy: 0.5000 - 35ms/epoch - 35ms/step
Epoch 3/5
1/1 - 0s - loss: 1113.9016 - accuracy: 0.5000 - 35ms/epoch - 35ms/step
Epoch 4/5
1/1 - 0s - loss: 126.5820 - accuracy: 0.5000 - 31ms/epoch - 31ms/step
Epoch 5/5
1/1 - 0s - loss: 194.5790 - accuracy: 0.5455 - 30ms/epoch - 30ms/step


...

1/1 [==============================] - 0s 164ms/step - loss: 13.8977 - accuracy: 0.5000
Epoch 1/5
1/1 - 1s - loss: 13.2611 - accuracy: 0.4545 - 526ms/epoch - 526ms/step
Epoch 2/5
1/1 - 0s - loss: 861.7039 - accuracy: 0.5455 - 35ms/epoch - 35ms/step
Epoch 3/5
1/1 - 0s - loss: 244.2320 - accuracy: 0.5455 - 35ms/epoch - 35ms/step
Epoch 4/5
1/1 - 0s - loss: 67.8657 - accuracy: 0.5455 - 29ms/epoch - 29ms/step
Epoch 5/5
1/1 - 0s - loss: 8.6044 - accuracy: 0.5000 - 29ms/epoch - 29ms/step
1/1 [==============================] - 0s 124ms/step - loss: 0.7494 - accuracy: 0.6818


 67%|██████▋   | 2/3 [00:09<00:04,  4.26s/it]

Epoch 1/5
1/1 - 1s - loss: 21.6093 - accuracy: 0.4091 - 537ms/epoch - 537ms/step
Epoch 2/5
1/1 - 0s - loss: 267.2184 - accuracy: 0.5000 - 34ms/epoch - 34ms/step
Epoch 3/5
1/1 - 0s - loss: 147.2806 - accuracy: 0.5000 - 33ms/epoch - 33ms/step
Epoch 4/5
1/1 - 0s - loss: 27.7311 - accuracy: 0.4091 - 31ms/epoch - 31ms/step
Epoch 5/5
1/1 - 0s - loss: 0.8167 - accuracy: 0.6364 - 31ms/epoch - 31ms/step
1/1 [==============================] - 0s 131ms/step - loss: 0.7036 - accuracy: 0.5000
Epoch 1/5
1/1 - 1s - loss: 19.4549 - accuracy: 0.5000 - 684ms/epoch - 684ms/step
Epoch 2/5
1/1 - 0s - loss: 1177.3665 - accuracy: 0.5000 - 37ms/epoch - 37ms/step
Epoch 3/5
1/1 - 0s - loss: 621.4453 - accuracy: 0.5000 - 38ms/epoch - 38ms/step
Epoch 4/5
1/1 - 0s - loss: 245.0368 - accuracy: 0.5000 - 34ms/epoch - 34ms/step
Epoch 5/5
1/1 - 0s - loss: 17.0015 - accuracy: 0.5000 - 32ms/epoch - 32ms/step
1/1 [==============================] - 0s 125ms/step - loss: 11.1063 - accuracy: 0.5000
Epoch 1/5
1/1 - 1s - loss:


 accuracy 0.6818181872367859
start_point = {'keras.compile.0': {'loss': ['binary_crossentropy'], 'optimizer': ['adam']}, 'keras.fit.0': {'epochs': [5], 'batch_size': [32], 'verbose': [2]}, 'keras.layers.Conv2D.1': {'filters': [32], 'kernel_size': [(3, 3)], 'activation': ['relu'], 'input_shape': [(256, 256, 3)]}, 'keras.layers.MaxPooling2D.2': {'pool_size': [(2, 2)]}, 'keras.layers.Conv2D.3': {'filters': [16], 'kernel_size': [(3, 3)], 'activation': ['relu']}, 'keras.layers.MaxPooling2D.4': {'pool_size': [(2, 2)]}, 'keras.layers.Flatten.5': {}, 'keras.layers.Dense.6': {'units': [30], 'activation': ['relu']}, 'keras.layers.Dropout.7': {'rate': [0.2]}, 'keras.layers.Dense.8': {'units': [2], 'activation': ['softmax']}}
1/1 [==============================] - 0s 58ms/step - loss: 0.6931 - accuracy: 0.5000
time: 37.81761717796326


1/1 [==============================] - 0s 28ms/step - loss: 0.6931 - accuracy: 0.5000
test score: 0.5
